Imports

In [1]:
import pandas as pd
import requests

In [13]:
def null_check(df):
    print('Shape: ', df.shape)
    print('\n', df.isna().sum())

Drivers

In [6]:
url = 'http://ergast.com/api/f1/drivers.json?limit=1000'
r = requests.get(url)
drivers = r.json()
df_drivers = pd.DataFrame(drivers['MRData']['DriverTable']['Drivers'])

In [14]:
null_check(df_drivers)

Shape:  (854, 8)

 driverId             0
url                  0
givenName            0
familyName           0
dateOfBirth          0
nationality          0
permanentNumber    803
code               757
dtype: int64


In [19]:
# recent drivers have both fields
df_drivers[df_drivers['permanentNumber'].notna()].head(10)

,driverId,url,givenName,familyName,dateOfBirth,nationality,permanentNumber,code
8,aitken,http://en.wikipedia.org/wiki/Jack_Aitken,Jack,Aitken,1995-09-23,British,89,AIT
10,albon,http://en.wikipedia.org/wiki/Alexander_Albon,Alexander,Albon,1996-03-23,Thai,23,ALB
16,alonso,http://en.wikipedia.org/wiki/Fernando_Alonso,Fernando,Alonso,1981-07-29,Spanish,14,ALO
75,jules_bianchi,http://en.wikipedia.org/wiki/Jules_Bianchi,Jules,Bianchi,1989-08-03,French,17,BIA
96,bottas,http://en.wikipedia.org/wiki/Valtteri_Bottas,Valtteri,Bottas,1989-08-28,Finnish,77,BOT
131,button,http://en.wikipedia.org/wiki/Jenson_Button,Jenson,Button,1980-01-19,British,22,BUT
158,chilton,http://en.wikipedia.org/wiki/Max_Chilton,Max,Chilton,1991-04-21,British,4,CHI
245,ericsson,http://en.wikipedia.org/wiki/Marcus_Ericsson,Marcus,Ericsson,1990-09-02,Swedish,9,ERI
267,pietro_fittipaldi,http://en.wikipedia.org/wiki/Pietro_Fittipaldi,Pietro,Fittipaldi,1996-06-25,Brazilian,51,FIT
302,gasly,http://en.wikipedia.org/wiki/Pierre_Gasly,Pierre,Gasly,1996-02-07,French,10,GAS


Seasons

In [20]:
url = 'http://ergast.com/api/f1/seasons.json?limit=1000'
r = requests.get(url)

In [25]:
seasons = r.json()
df_seasons = pd.DataFrame(seasons['MRData']['SeasonTable']['Seasons'])

In [28]:
seasons['MRData']['SeasonTable']['Seasons'][-1]

{'season': '2022',
 'url': 'http://en.wikipedia.org/wiki/2022_Formula_One_World_Championship'}

In [26]:
df_seasons.head()

,season,url
0,1950,http://en.wikipedia.org/wiki/1950_Formula_One_...
1,1951,http://en.wikipedia.org/wiki/1951_Formula_One_...
2,1952,http://en.wikipedia.org/wiki/1952_Formula_One_...
3,1953,http://en.wikipedia.org/wiki/1953_Formula_One_...
4,1954,http://en.wikipedia.org/wiki/1954_Formula_One_...


Qualifying

In [66]:
url = 'http://ergast.com/api/f1/2009/qualifying.json?limit=10000'
r = requests.get(url)
quali = r.json()

In [67]:
df_quali = pd.DataFrame(quali['MRData']['RaceTable']['Races'])

In [68]:
df_quali.head()

,season,round,url,raceName,Circuit,date,time,QualifyingResults
0,2009,1,http://en.wikipedia.org/wiki/2009_Australian_G...,Australian Grand Prix,"{'circuitId': 'albert_park', 'url': 'http://en...",2009-03-29,06:00:00Z,"[{'number': '22', 'position': '1', 'Driver': {..."
1,2009,2,http://en.wikipedia.org/wiki/2009_Malaysian_Gr...,Malaysian Grand Prix,"{'circuitId': 'sepang', 'url': 'http://en.wiki...",2009-04-05,09:00:00Z,"[{'number': '22', 'position': '1', 'Driver': {..."
2,2009,3,http://en.wikipedia.org/wiki/2009_Chinese_Gran...,Chinese Grand Prix,"{'circuitId': 'shanghai', 'url': 'http://en.wi...",2009-04-19,07:00:00Z,"[{'number': '15', 'position': '1', 'Driver': {..."
3,2009,4,http://en.wikipedia.org/wiki/2009_Bahrain_Gran...,Bahrain Grand Prix,"{'circuitId': 'bahrain', 'url': 'http://en.wik...",2009-04-26,12:00:00Z,"[{'number': '9', 'position': '1', 'Driver': {'..."
4,2009,5,http://en.wikipedia.org/wiki/2009_Spanish_Gran...,Spanish Grand Prix,"{'circuitId': 'catalunya', 'url': 'http://en.w...",2009-05-10,12:00:00Z,"[{'number': '22', 'position': '1', 'Driver': {..."


In [69]:
root = quali['MRData']['RaceTable']['Races']
len(root)

17

In [64]:
for _ in root:
    print(_['season'])

1994
1994
1994
1994
1994
1994
1994
1994
1994
1994
1994
1994
1994
1994
1994
1995
1995
1995
1995
1995
1995
1995
1995
1995
1995
1995
1995
1995
1995
1995
1995
1995
1996
1996
1996
1996
1996
1996
1996
1997
1997


In [49]:
root[-1]['QualifyingResults'][0]

{'number': '3',
 'position': '1',
 'Driver': {'driverId': 'villeneuve',
  'code': 'VIL',
  'url': 'http://en.wikipedia.org/wiki/Jacques_Villeneuve',
  'givenName': 'Jacques',
  'familyName': 'Villeneuve',
  'dateOfBirth': '1971-04-09',
  'nationality': 'Canadian'},
 'Constructor': {'constructorId': 'williams',
  'url': 'http://en.wikipedia.org/wiki/Williams_Grand_Prix_Engineering',
  'name': 'Williams',
  'nationality': 'British'},
 'Q1': '1:16.004'}

In [41]:
df_quali['Circuit']

0     {'circuitId': 'interlagos', 'url': 'http://en....
1     {'circuitId': 'okayama', 'url': 'http://en.wik...
2     {'circuitId': 'imola', 'url': 'http://en.wikip...
3     {'circuitId': 'catalunya', 'url': 'http://en.w...
4     {'circuitId': 'villeneuve', 'url': 'http://en....
5     {'circuitId': 'magny_cours', 'url': 'http://en...
6     {'circuitId': 'silverstone', 'url': 'http://en...
7     {'circuitId': 'hockenheimring', 'url': 'http:/...
8     {'circuitId': 'hungaroring', 'url': 'http://en...
9     {'circuitId': 'spa', 'url': 'http://en.wikiped...
10    {'circuitId': 'monza', 'url': 'http://en.wikip...
11    {'circuitId': 'estoril', 'url': 'http://en.wik...
12    {'circuitId': 'jerez', 'url': 'http://en.wikip...
13    {'circuitId': 'suzuka', 'url': 'http://en.wiki...
14    {'circuitId': 'adelaide', 'url': 'http://en.wi...
15    {'circuitId': 'interlagos', 'url': 'http://en....
16    {'circuitId': 'galvez', 'url': 'http://en.wiki...
17    {'circuitId': 'imola', 'url': 'http://en.w